In [64]:
import pandas as pd
import numpy as np
import json
import re
import reverse_geocoder as rg

In [58]:
df = pd.read_pickle('./Data/business.pkl')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

In [59]:
df = df.loc[:,~df.columns.duplicated()]

In [17]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [62]:
df['categories'].unique()

array(['Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists',
       'Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services',
       'Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores',
       ...,
       'Shopping, Jewelry, Piercing, Toy Stores, Beauty & Spas, Accessories, Fashion',
       'Fitness/Exercise Equipment, Eyewear & Opticians, Shopping, Sporting Goods, Bikes',
       'Beauty & Spas, Permanent Makeup, Piercing, Tattoo'], dtype=object)

### Asignacion Estados por coordenadas

In [66]:
df['latitude'] = pd.to_numeric(df['latitude'])
df['longitude'] = pd.to_numeric(df['longitude'])

In [67]:
coordinates = list(zip(df['latitude'], df['longitude']))
results = rg.search(coordinates)

Loading formatted geocoded file...


In [68]:
states = [result['admin1'] for result in results]
df['states'] = states

In [71]:
df.drop('state', axis=1, inplace=True)

### Categorias Restaurant - Hotels

In [77]:
df['categories'].fillna('', inplace=True)

df = df[df['categories'].str.contains('restaurant|restaurants|hotel|hotels', case=False)]

In [87]:
df.sample(5)

,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,states
137588,SUL_XDzGnnccUgHqnix1Ng,The Slider House,1907 Division St,Nashville,37203,36.149811,-86.796336,3.5,245,1,"{'Caters': 'True', 'WiFi': ''free'', 'Business...","American (New), Food, Musicians, Sports Bars, ...","{'Monday': '11:0-0:0', 'Tuesday': '11:0-0:0', ...",Tennessee
70272,w9WLlcY_mziJ6qAB6WtpsA,The Flying V Bar & Grill,7000 N Resort Dr,Tucson,85750,32.327823,-110.850591,4.0,152,1,"{'RestaurantsGoodForGroups': 'True', 'Caters':...","Restaurants, Hotels, American (Traditional), N...","{'Wednesday': '17:0-21:0', 'Thursday': '17:0-2...",Arizona
23405,bSehfDoKVeCXIcwcFwX38w,Dumpling House,925 Arch St,Philadelphia,19107,39.953814,-75.155838,3.5,18,0,"{'OutdoorSeating': 'False', 'Ambience': '{'tou...","Asian Fusion, Restaurants","{'Monday': '10:30-22:30', 'Tuesday': '10:30-22...",Pennsylvania
112070,uR8Ew5NDKj2ZPVPv-3EYgg,Domino's Pizza,11224 Boyette Rd,Riverview,33569,27.854320,-82.316849,2.0,46,1,"{'RestaurantsPriceRange2': '1', 'GoodForKids':...","Sandwiches, Restaurants, Chicken Wings, Pizza","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",Florida
35768,DKcFxSfjLvSXRB9rOKE8jw,KFC,9260 Ellerslie Road SW,Edmonton,T6X 0K6,53.425397,-113.476620,1.5,9,1,"{'HasTV': 'False', 'RestaurantsPriceRange2': '...","Fast Food, Mexican, Restaurants, American (Tra...","{'Monday': '10:30-23:30', 'Tuesday': '10:30-23...",Alberta


## Asignacion de categorias y simplificar la data

In [89]:
keyword_category_mapping = {
    'Barbeque': 'Restaurant American Food Nightlife - Dance Clubs',
    '(New)': 'Restaurant American Food Nightlife - Dance Clubs',
    'Thai': 'Restaurant Thai',
    'Colombian': 'Restaurant Colombian',
    'Peruvian': 'Restaurant Peruvian',
    'Street': ' Food Street',
    'Mediterranean': 'Restaurant Mediterranean - Middle Eastern',
    'Indian': 'Restaurant Indian - Food Pakistani',
    'Ice Cream': 'Restaurant Ice Cream & Frozen Yogurt',
    'Vietnamese': 'Restaurant Vietnamese',
    'Bakeries': 'Restaurant Coffee & Tea',
    'Coffee': 'Restaurant Coffee & Tea',
    'Brazilian': 'Restaurant Brazilian Steakhouses',
    'Chicken': 'Restaurant Chicken Wings - Fast Food',
    'Vegetarian': 'Restaurant Vegetarian Traditional',
    'Japanese': 'Restaurant Japanese',
    'Sandwiches': 'Restaurant Sandwiches Fast Food',
    'Nightlife': 'Restaurant American Food Nightlife - Dance Clubs',
    'Greek': 'Restaurant Greek Modern Europe',
    'Breakfast & Brunch': 'Restaurant Breakfast & Brunch American',
    'Italian': 'Restaurant Italian',
    'Cafes': 'Restaurant Coffee & Tea',
    'Burgers':  'Restaurant Burgers Fast Food',
    'Chinese': 'Restaurant Chinese',
    'Pizza': 'Restaurant Pizza Fast Food',
    'Pet Stores': 'Restaurant Friendly Pets',
    'Event Planning & Services': 'Restaurant Event Planning & Services',
    'American': 'Restaurant Breakfast & Brunch American',
    'Local Flavor':'Restaurant Breakfast & Brunch American',
    'Cheesesteaks': 'Restaurant Cheesesteaks',
    'Hot Dogs': 'Restaurant  Hot Dogs  Fast Food',
    'Caribbean': 'Restaurant Caribbean', 
    'Asian': 'Restaurant Asian',
    'Korean': 'Restaurant Korean',
    'Wine': 'Restaurant American Food Nightlife - Dance Clubs',
    'Breweries': 'Restaurant American Food Nightlife - Dance Clubs',
    'Southern': 'Restaurant Southern',
    'Filipino': 'Restaurant Filipino',
    'Comfort': 'Restaurant American Food Nightlife - Dance Clubs',
    'Venezuelan': 'Restaurant Venezuelan',
    'Salad': 'Restaurant American Food Nightlife - Dance Clubs',
    'Cafeteria': 'Restaurant Coffee & Tea',
    'Fish': 'Restaurant Fish & Seafood',
    'Seafood': 'Restaurant Seafood',
    'Creperies': 'Restaurant Creperies',
    'Juice Bars & Smoothies': 'Restaurant Juice Bars & Smoothies',
    'Donuts': 'Restaurant Coffee & Tea',
    'Hawaiian': 'Restaurant Hawaiian',
    'Mexican': 'Restaurant Mexican',
    'Grocery': 'Food Ethnic International',
    'Cuban': 'Restaurant Cuban',
    'Sushi': 'Restaurant Japanese',
    'Afghan': 'Restaurant Indian - Food Pakistani',
    'British': 'Restaurant Greek Modern Europe',
    'Cheese': 'Restaurant Cheesesteaks',
    'Middle Eastern': 'Restaurant Middle Eastern'
}

In [90]:
for keyword, category in keyword_category_mapping.items():
    keyword_mask = df['categories'].str.contains(keyword, case=False)
    total_keyword_rows = keyword_mask.sum()
    print(f"Total de filas con '{keyword}': {total_keyword_rows}")
    df.loc[keyword_mask, 'categories'] = category

Total de filas con 'Barbeque': 1694
Total de filas con '(New)': 6216
Total de filas con 'Thai': 951


/tmp/ipykernel_22421/1774109774.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  keyword_mask = df['categories'].str.contains(keyword, case=False)


Total de filas con 'Colombian': 64
Total de filas con 'Peruvian': 77
Total de filas con 'Street': 213
Total de filas con 'Mediterranean': 1090
Total de filas con 'Indian': 791
Total de filas con 'Ice Cream': 988
Total de filas con 'Vietnamese': 736
Total de filas con 'Bakeries': 1654
Total de filas con 'Coffee': 4731
Total de filas con 'Brazilian': 62
Total de filas con 'Chicken': 2856
Total de filas con 'Vegetarian': 542
Total de filas con 'Japanese': 1548
Total de filas con 'Sandwiches': 4838
Total de filas con 'Nightlife': 12472
Total de filas con 'Greek': 252
Total de filas con 'Breakfast & Brunch': 2183
Total de filas con 'Italian': 2245
Total de filas con 'Cafes': 413
Total de filas con 'Burgers': 1733
Total de filas con 'Chinese': 2313
Total de filas con 'Pizza': 2449
Total de filas con 'Pet Stores': 5
Total de filas con 'Event Planning & Services': 3502
Total de filas con 'American': 15963
Total de filas con 'Local Flavor': 143
Total de filas con 'Cheesesteaks': 39
Total de fil

### Eliminacion Columnas 

In [91]:
df.drop('is_open', axis=1, inplace=True)

/tmp/ipykernel_22421/3072124205.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('is_open', axis=1, inplace=True)


### Transformaciones

In [102]:
df['hours']

0        {'Friday': '7:0-21:0', 'Monday': '7:0-20:0', '...
1        {'Friday': '9:0-0:0', 'Monday': '0:0-0:0', 'Sa...
2                                                     None
3        {'Friday': '6:0-16:0', 'Monday': '0:0-0:0', 'S...
4        {'Friday': '11:0-14:0', 'Monday': '11:0-14:0',...
                               ...                        
57717    {'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...
57718    {'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...
57719    {'Friday': '11:0-2:0', 'Monday': '11:0-22:0', ...
57720    {'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...
57721    {'Friday': '11:0-22:0', 'Monday': '11:0-22:0',...
Name: hours, Length: 57722, dtype: object

In [103]:
data_types = df['hours'].map(type)
data_types

0            <class 'dict'>
1            <class 'dict'>
2        <class 'NoneType'>
3            <class 'dict'>
4            <class 'dict'>
                ...        
57717        <class 'dict'>
57718        <class 'dict'>
57719        <class 'dict'>
57720        <class 'dict'>
57721        <class 'dict'>
Name: hours, Length: 57722, dtype: object

In [106]:
df['day'] = None
df['hour'] = None

for index, row in df.iterrows():
    if isinstance(row['hours'], dict):
        days = []
        hours = []
        for day, hour in row['hours'].items():
            days.append(day)
            hours.append(hour if hour is not None else '')  
        
        df.at[index, 'day'] = ', '.join(days)
        df.at[index, 'hour'] = ', '.join(hours)

In [108]:
df.drop('hours', axis=1, inplace=True)

In [118]:
df['attributes']

0        {'AcceptsInsurance': None, 'AgesAllowed': None...
1        {'AcceptsInsurance': None, 'AgesAllowed': None...
2        {'AcceptsInsurance': None, 'AgesAllowed': None...
3        {'AcceptsInsurance': None, 'AgesAllowed': None...
4        {'AcceptsInsurance': None, 'AgesAllowed': None...
                               ...                        
57717    {'AcceptsInsurance': None, 'AgesAllowed': None...
57718    {'AcceptsInsurance': None, 'AgesAllowed': None...
57719    {'AcceptsInsurance': None, 'AgesAllowed': None...
57720    {'AcceptsInsurance': None, 'AgesAllowed': None...
57721    {'AcceptsInsurance': None, 'AgesAllowed': None...
Name: attributes, Length: 57722, dtype: object

In [119]:
data_types = df['attributes'].map(type)
data_types

0        <class 'dict'>
1        <class 'dict'>
2        <class 'dict'>
3        <class 'dict'>
4        <class 'dict'>
              ...      
57717    <class 'dict'>
57718    <class 'dict'>
57719    <class 'dict'>
57720    <class 'dict'>
57721    <class 'dict'>
Name: attributes, Length: 57722, dtype: object

In [123]:
df['attribute'] = None

for index, row in df.iterrows():
    if isinstance(row['attributes'], dict):
        attributes_text = []
        for key, value in row['attributes'].items():
            if value is not None:
                attributes_text.append(f"{key}: {value}")
            else:
                attributes_text.append(key)
        
        df.at[index, 'attribute'] = ', '.join(attributes_text)

In [135]:
def split_text(text):
    if isinstance(text, str): 
        return re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    else:
        return text

df['attribute'] = df['attribute'].apply(split_text)

In [125]:
df.drop('attributes', axis=1, inplace=True)

In [130]:
column_order = ['business_id', 'name', 'address','city','states','postal_code','latitude','longitude','stars','review_count','attribute','categories','day','hour']
df = df.reindex(columns=column_order)

In [155]:
df['longitude']

0        -75.155564
1        -87.058943
2        -90.321087
3        -86.768170
4        -82.456320
            ...    
57717   -116.284689
57718   -111.093308
57719    -75.158665
57720    -75.427725
57721   -113.675999
Name: longitude, Length: 57722, dtype: float64

In [157]:
df['longitude'] = df['longitude'].astype(str)

df['latitude'] = df['latitude'].astype(str)

In [158]:
df.isnull().sum()

business_id        0
name               0
address            0
city               0
states             0
postal_code        0
latitude           0
longitude          0
stars              0
review_count       0
attribute       2125
categories         0
day             8476
hour            8476
dtype: int64

In [169]:
df.sample(5)

,business_id,name,address,city,states,postal_code,latitude,longitude,stars,review_count,attribute,categories,day,hour
8808,oipTcwrQb6jQy0VrqpFumQ,Seoul Garden,10441 Olive Blvd,Creve Coeur,Missouri,63141,38.674089,-90.410002,3.0,125,"Accepts Insurance, Ages Allowed, Alcohol: 'bee...",Restaurant Breakfast & Brunch American,"Friday, Monday, Saturday, Sunday, Thursday, Tu...","12:0-19:0, 11:0-21:0, 11:0-21:0, 12:0-21:0, 11..."
54372,r6MBlUIS8_U1JINh8HJZTw,Ciao Pizza & Pasta,3218 Kirkwood Hwy,Wilmington,Delaware,19808,39.736062746900004,-75.6242229672,3.5,23,"Accepts Insurance, Ages Allowed, Alcohol: 'non...",Restaurant Pizza Fast Food,"Friday, Monday, Saturday, Sunday, Thursday, Tu...","11:0-21:0, 11:0-17:0, 11:0-21:0, 12:0-18:0, 11..."
55651,W1aLlzTpnF6ue27JDTiTtw,Los Arroyos,"11503 Springmill Rd, Ste 100",Carmel,Indiana,46032,39.954837,-86.164557,4.0,227,"Accepts Insurance, Ages Allowed, Alcohol: u'fu...",Restaurant Breakfast & Brunch American,"Friday, Monday, Saturday, Sunday, Thursday, Tu...","11:0-22:0, 0:0-0:0, 11:0-22:0, 11:0-15:0, 11:0..."
40030,RF2SocELaoYkXnZhxRcX0g,Gallagher's Restaurant,114 W Mill St,Waterloo,Illinois,62298,38.3358432,-90.1511628,4.0,162,"Accepts Insurance, Ages Allowed, Alcohol: u'fu...",Restaurant Breakfast & Brunch American,"Friday, Monday, Saturday, Sunday, Thursday, Tu...","11:0-22:0, , 11:0-22:0, 11:30-19:0, 11:0-21:0,..."
56767,UzRMlnXQKlWa16_spVHHXw,Joselito's Carnes Asadas,927 Bell Rd,Nashville,Tennessee,37013,36.0494217586,-86.651636131,3.5,10,"Accepts Insurance, Ages Allowed, Alcohol: u'fu...",Restaurant Mexican,None,None


In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57722 entries, 0 to 57721
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   57722 non-null  object 
 1   name          57722 non-null  object 
 2   address       57722 non-null  object 
 3   city          57722 non-null  object 
 4   states        57722 non-null  object 
 5   postal_code   57722 non-null  object 
 6   latitude      57722 non-null  object 
 7   longitude     57722 non-null  object 
 8   stars         57722 non-null  float64
 9   review_count  57722 non-null  int64  
 10  attribute     55597 non-null  object 
 11  categories    57722 non-null  object 
 12  day           49246 non-null  object 
 13  hour          49246 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 6.2+ MB


In [167]:
df.to_parquet('business.parquet',index=False)